In [0]:
%run /Workspace/Users/pragatitak007@gmail.com/configs

In [0]:
results_df_silver = spark.read.parquet(silver_path + '/results')
drivers_df_silver = spark.read.parquet(silver_path + '/drivers')
constructors_df_silver = spark.read.parquet(silver_path + '/constructors')

/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: Could not open file </Workspace/Users/pragatitak007@gmail.com/f1_project_notebook_bronze_ingestion> for safe execution.
  warn('Could not open file <%s> for safe execution.' % fname)
/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: Could not open file </Workspace/Users/pragatitak007@gmail.com/f1_project_notebook_silver> for safe execution.
  warn('Could not open file <%s> for safe execution.' % fname)


In [0]:
from pyspark.sql.functions import *

race_results_df = (results_df_silver
                       .join(drivers_df_silver, on='driver_id') 
                       .join(constructors_df_silver, on='constructor_id')
                       .select(
                           drivers_df_silver.name.alias('Driver'),
                           constructors_df_silver.name.alias('Team'),
                           results_df_silver.position_order.alias('Position'),
                           results_df_silver.points.alias('Points')
                           
                       ))

#display(race_results_df)


standings_df = (race_results_df
                .groupBy('Driver','Team')
                .agg(
                    sum('Points').alias('Points'),
                    count(when(col('Position') == 1, True)).alias('Wins'))
                .select(col('Driver'), col('Team'), col('Points'), col('Wins'))
                .orderBy(col("Points").desc(), col("Wins").desc()))

display(standings_df)

standings_df.write.mode('overwrite').parquet(gold_path + '/driver_standings')

Driver,Team,Points,Wins
Lewis Hamilton,Mercedes,2865.0,74
Sebastian Vettel,Red Bull,1577.0,38
Nico Rosberg,Mercedes,1519.0,23
Sebastian Vettel,Ferrari,1400.0,14
Fernando Alonso,Ferrari,1190.0,11
Valtteri Bottas,Mercedes,1101.0,9
Max Verstappen,Red Bull,1100.0,10
Kimi Räikkönen,Ferrari,1080.0,10
Michael Schumacher,Ferrari,1066.0,72
Mark Webber,Red Bull,978.5,9


In [0]:
team_standings_df = (race_results_df
                     .groupBy('Team')
                     .agg(
                         sum('Points').alias('Points'),
                    count(when(col('Position') == 1, True)).alias('Wins'))
                     .select(col('Team'), col('Points'), col('Wins'))
                     .orderBy(col("Points").desc(), col("Wins").desc()))

display(team_standings_df)

team_standings_df.write.mode('overwrite').parquet(gold_path + '/team_standings')

Team,Points,Wins
Ferrari,9292.77,239
Mercedes,5824.14,115
McLaren,5723.5,178
Red Bull,5043.5,64
Williams,3567.0,114
Renault,1777.0,35
Force India,1098.0,0
Team Lotus,995.0,45
Benetton,861.5,27
Tyrrell,711.0,23
